In [3]:
print("\nRunning Update HOTzone Python Script\n\n\n")
print("Importing geopandas site package...\n")
import geopandas as gpd
import sys
import os
import imp
import time
import traceback

TAZ_shp = r'inputs-04-14\\TAZ.shp'
Scenario_Link = r'inputs-04-14\\C1_Link.shp'
Scenario_Node = r'inputs-04-14\\C1_Node.shp'
UsedZones = '3629'
temp_folder = r'temp_gpd-test'
log = os.path.join(temp_folder,r'LogFile_UpdateLinkNodeTAZID.txt')

# Print key variabls to screen
print("TAZ_shp: \n    "           + TAZ_shp           + "\n")
print("Scenario_Link: \n    "     + Scenario_Link     + "\n")
print("Scenario_Node: \n    "     + Scenario_Node     + "\n")
print("UsedZones: \n    "         + str(UsedZones)    + "\n")
print("Temp Folder:  \n    "      + temp_folder       + "\n")

# Open log file
logFile = open(log, 'w')
logFile.write(TAZ_shp+'\n')
logFile.write(Scenario_Link+'\n')
logFile.write(Scenario_Node+'\n')
logFile.write(str(UsedZones)+'\n')
logFile.write(temp_folder+'\n')

#raw_input()


## Define Variables
# Variables: Input
taz_shp           = str(TAZ_shp)
link_shp          = str(Scenario_Link)
node_shp          = str(Scenario_Node)
UsedZones         = str(UsedZones)
temp_folder       = str(temp_folder)

# Variables: Intermediate
link_taz_sj       = os.path.join(temp_folder, "Link_TAZ_SJ_deleteTemp.shp")
node_taz_sj       = os.path.join(temp_folder, "Node_TAZ_SJ_deleteTemp.shp")

# Variables: Output
out_link          = os.path.join(temp_folder, "C1_Link_TAZID.shp")
out_node          = os.path.join(temp_folder, "C1_Node_TAZID.shp")
out_link_mp       = os.path.join(temp_folder, "C1_Link_Midponts.shp")

# Codeblock to calculate TAZID
def calcTAZID_Node(tazid, node, global_n):
  if node <= int(global_n):
    return node
  else:
    return tazid

def calcTAZID_Link(tazid, aField, bField, global_n):
    if int(aField) <= int(global_n):
        return aField
    elif int(bField) <= int(global_n):
        return bField
    else:
        return tazid


Running Update HOTzone Python Script



Importing geopandas site package...

TAZ_shp: 
    inputs-04-14\\TAZ.shp

Scenario_Link: 
    inputs-04-14\\C1_Link.shp

Scenario_Node: 
    inputs-04-14\\C1_Node.shp

UsedZones: 
    3629

Temp Folder:  
    temp_gpd-test



In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

## TagLinksWithTAZID()

In [5]:
print("\n\nImporting Highway Link data...")
gdf_link = gpd.read_file(link_shp)




Importing Highway Link data...


In [6]:
print("\nCalculating Highway Link distance (in miles) and updating DISTANCE field...")
gdf_link["DISTANCE"] = gdf_link.geometry.length / 1609.34  # Conversion from meters to miles  (DO I NEED THIS CONVERSION?)


Calculating Highway Link distance (in miles) and updating DISTANCE field...


In [7]:
print("\nCalculating Highway Link midpoint coord and updating X_MID & Y_MID fields...")
gdf_link["X_MID"] = gdf_link.geometry.interpolate(0.5, normalized=True).x
gdf_link["Y_MID"] = gdf_link.geometry.interpolate(0.5, normalized=True).y


Calculating Highway Link midpoint coord and updating X_MID & Y_MID fields...


In [8]:
print("\nMaking new point shapefile from Highway Link midpoint...")
gdf_midpoints = gdf_link.copy()
gdf_midpoints = gdf_midpoints.set_geometry(gdf_midpoints.geometry.centroid)
gdf_midpoints.to_file(out_link_mp)
gdf_midpoints


Making new point shapefile from Highway Link midpoint...


,A,B,DISTANCE,DISTEXCEPT,EXTERNAL,LINKID,FLG_NEWNET,MAG_LINK,STREET,HOTZN,SEL_LINK,ONEWAY,DIRECTION,SFAC_BASE,SFAC_FUT,CFAC_BASE,CFAC_FUT,TRKRST2015,TRKRST2023,TAZID,SEGID,SEGEX_RTP,SEGEX_NEED,LN_2015,LN_2019,LN_2021,LN_2023,LN_2024,LN_2028,LN23_32,LN23_42,LN23_50,LN23_32UF,LN23_42UF,LN23_50UF,FT_2015,FT_2019,FT_2021,FT_2023,FT_2024,FT_2028,FT23_32,FT23_42,FT23_50,FT23_32UF,FT23_42UF,FT23_50UF,TSPD_2015,TSPD_2019,TSPD_2021,TSPD_2023,TSPD_2024,TSPD_2028,TSPD23_32,TSPD23_42,TSPD23_50,TSPD23_32U,TSPD23_42U,TSPD23_50U,TSPD_AVE,TRNSPD_PTC,TRNSPD_MIS,TRNSPD_MED,TRNSPD_HIG,TRNSPD_HII,HOT_2015,HOT_2019,HOT_2021,HOT_2023,HOT_2024,HOT_2028,HOT23_32,HOT23_42,HOT23_50,HOT23_32UF,HOT23_42UF,HOT23_50UF,REL_2015,REL_2019,REL_2021,REL_2023,REL_2024,REL_2028,REL23_32,REL23_42,REL23_50,REL23_32UF,REL23_42UF,REL23_50UF,OP_2019,OP_2021,OP_2023,OP_2024,OP_2028,OP23_32,OP23_42,OP23_50,OP23_32UF,OP23_42UF,OP23_50UF,GIS23_32,GIS23_42,GIS23_50,SCRN_LRG,SCRN_SML,FC_ID,SA_ID,LN19_30,LN19_40,LN19_50,LN19_30UF,LN19_40UF,LN19_50UF,LN19_50NNR,FT19_30,FT19_40,FT19_50,FT19_30UF,FT19_40UF,FT19_50UF,FT19_50NNR,TSPD19_30,TSPD19_40,TSPD19_50,TSPD19_30U,TSPD19_40U,TSPD19_50U,HOT19_30,HOT19_40,HOT19_50,REL19_30,REL19_40,REL19_50,OP19_30,OP19_40,OP19_50,OP19_30UF,OP19_40UF,OP19_50UF,OP19_50NNR,GIS19_30,GIS19_40,GIS19_50,SORT_L,geometry,X_MID,Y_MID
0,1,29685,0.387704,0,0,1_29685,0,0,Cent,0,0,0,EB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,POINT (412206.563 4605795.562),412206.562800,4.605796e+06
1,2,29685,0.108710,0,0,2_29685,0,0,Cent,0,0,0,WB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,POINT (412559.563 4605981.062),412559.562800,4.605981e+06
2,3,29645,0.518017,0,0,3_29645,0,0,Cent,0,0,0,EB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,POINT (409312.708 4603276.537),409312.707700,4.603277e+06
3,4,29671,0.254196,0,0,4_29671,0,0,Cent,0,0,0,WB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,POINT (409846.556 4604799.603),409846.555850,4.604800e+06
4,5,29680,0.294677,0,0,5_29680,0,0,Cent,0,0,0,NB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,POINT (413438.000 4603833.000),413438.000000,4.603833e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56472,95064,95025,1.515087,0,0,95064_95025,23,1,Sixth Water Creek,0,0,2,WB,0.0,0.0,0.0,0.0,0,0,0,None,None,Non

## geopandas.sjoin_nearest test

In [25]:
#gdf_nearestn0 = gdf_nearest.loc[gdf_nearest['nearest_dist']>0]
#gdf_nearestn0[['A','B','LINKID','DIRECTION','TAZID_left','geometry','X_MID','Y_MID','TAZID_right','nearest_dist']]
##repeats
#gdf_nearest_small = gdf_nearest[['A','B','LINKID','DIRECTION','TAZID_left','geometry','X_MID','Y_MID','TAZID_right','nearest_dist']]
#duplicates = gdf_nearest_small[gdf_nearest_small.duplicated(subset=['LINKID'], keep=False)]
#
## Filter the DataFrame to include only rows with repeated values
#filtered_df = gdf_nearest_small[gdf_nearest_small['LINKID'].isin(duplicates['LINKID'])]
#filtered_df

,A,B,LINKID,DIRECTION,TAZID_left,geometry,X_MID,Y_MID,TAZID_right,nearest_dist
21139,24551,24651,24551_24651,NB,0,POINT (424012.500 4507888.000),424012.5,4507888.0,1449,0.0
21364,24651,24551,24651_24551,SB,0,POINT (424012.500 4507888.000),424012.5,4507888.0,1449,0.0
21139,24551,24651,24551_24651,NB,0,POINT (424012.500 4507888.000),424012.5,4507888.0,1451,0.0
21364,24651,24551,24651_24551,SB,0,POINT (424012.500 4507888.000),424012.5,4507888.0,1451,0.0


In [33]:
print("\nSpatial joining TAZ to Link midpoints (this may take a few minutes)...")
# read in taz shapefile and calculate which tazes are closest to each midpoint value
gdf_taz = gpd.read_file(taz_shp)
gdf_nearest = gpd.sjoin_nearest(gdf_midpoints,gdf_taz, distance_col = 'nearest_dist')



Spatial joining TAZ to Link midpoints (this may take a few minutes)...


C:\Users\cday\AppData\Local\Temp\ipykernel_14984\4022382687.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nearest.to_file(os.path.join(temp_folder, "test.shp"))


In [42]:
#drop duplicates where tazes are equidistant from the link midpoint by keeping the first occurence
second_occurences = gdf_nearest['LINKID'].duplicated(keep='first')
gdf_nearest_final = gdf_nearest[~second_occurences]
gdf_link_taz_sj = gdf_nearest_final.rename(columns={'TAZID_left':'TAZID','TAZID_right':'TAZID_1'})
gdf_link_taz_sj.to_file(link_taz_sj)

C:\Users\cday\AppData\Local\Temp\ipykernel_14984\1928136348.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_link_taz_sj.to_file(link_taz_sj)


In [37]:
drop_columns = ['TAZID_V832', 'SORT', 'CO_IDX', 'CO_TAZID', 'SUBAREAID', 'ACRES', 'DEVACRES', 'DEVPBLEPCT', 'X', 'Y', 'ADJ_XY', 'CO_FIPS', 'CO_NAME', 'CITY_FIPS', 'CITY_UGRC', 'CITY_NAME', 'DISTSUPER', 'DSUP_NAME', 'DISTLRG', 'DLRG_NAME', 'DISTMED', 'DMED_NAME', 'DISTSML', 'DSML_NAME', 'CBD', 'TERMTIME', 'PRKCSTPERM', 'PRKCSTTEMP', 'WALK100', 'ECOEDPASS', 'FREEFARE', 'REMM']

In [41]:
print("\nUpdating Highway Link TAZID...\n")
gdf_link_mp = gdf_link_taz_sj.copy()
gdf_link_mp["TAZID"] = gdf_link_mp.apply(lambda row: calcTAZID_Link(row["TAZID_1"], row["A"], row["B"], UsedZones), axis=1)
gdf_link_mp = gdf_link_mp.drop(columns=drop_columns).drop(columns={'nearest_dist','TAZID_1'})
gdf_link_mp = gdf_link_mp.sort_values(by='LINKID', ascending=True)
gdf_link_mp.to_file(out_link)


Updating Highway Link TAZID...



C:\Users\cday\AppData\Local\Temp\ipykernel_14984\3126247943.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_link_mp.to_file(out_link)


# TagNodesWithTAZID()

In [5]:
print("\n\nImporting Highway Node data...")
gdf_nodes = gpd.read_file(node_shp)



Importing Highway Node data...


In [21]:
print("\nSpatial joining TAZ to Highway Nodes (this may take a few minutes)...")

# read in taz shapefile and calculate which tazes are closest to each midpoint value
gdf_taz = gpd.read_file(taz_shp)
gdf_nodes_nearest_taz = gpd.sjoin_nearest(gdf_nodes,gdf_taz, distance_col = 'nearest_dist')

#drop duplicates where tazes are equidistant from the link midpoint by keeping the first occurence
second_node_occurences = gdf_nodes_nearest_taz['N'].duplicated(keep='first')
gdf_node_nearest_final = gdf_nodes_nearest_taz[~second_node_occurences]
gdf_node_taz_sj = gdf_node_nearest_final.rename(columns={'X_left':'X','Y_left':'Y','TAZID_left':'TAZID','TAZID_right':'TAZID_1','X_right':'X_1','Y_right':'Y_1'})
gdf_node_taz_sj = gdf_node_taz_sj.drop(columns={'nearest_dist'})
gdf_node_taz_sj.to_file(node_taz_sj)


Spatial joining TAZ to Highway Nodes (this may take a few minutes)...


C:\Users\cday\AppData\Local\Temp\ipykernel_25160\1871447323.py:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_node_taz_sj.to_file(node_taz_sj)


In [20]:
print("\nUpdating Highway Node TAZID...\n")
gdf_node_mp = gdf_node_taz_sj.copy()
gdf_node_mp['TAZID'] = gdf_node_mp.apply(lambda row: calcTAZID_Node(row['TAZID_1'], row['N'], UsedZones), axis = 1)
gdf_node_mp = gdf_node_mp.iloc[:,:43]
gdf_node_mp.to_file(out_node)


Updating Highway Node TAZID...

